In [7]:
!pip install -e .

Obtaining file:///content/mendez_juan_tp_dt
  Preparing metadata (setup.py) ... done
  Attempting uninstall: my_project
    Found existing installation: my_project 0.1
    Uninstalling my_project-0.1:
      Successfully uninstalled my_project-0.1
  Running setup.py develop for my_project


In [8]:
import torch
import torch.nn as nn
from transformers import GPT2Config, GPT2Model
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import pickle

# Configuración del dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# === CONFIGURACIÓN DEL DATASET ===
# Cambiar aquí para elegir Netflix o Goodreads
DATASET = 'netflix'  # o 'goodreads'

if DATASET == 'netflix':
    NUM_ITEMS = 752
    train_path = 'data/train/netflix8_train.df'
    test_path = 'data/test_users/netflix8_test.json'
else:
    NUM_ITEMS = 472
    train_path = 'data/train/goodreads8_train.df'
    test_path = 'data/test_users/goodreads8_test.json'

NUM_GROUPS = 8

Using device: cuda


In [9]:
# === 1. CARGAR DATOS ===
print("\n[1/5] Cargando datos...")

with open(train_path, 'rb') as f:
    df_train = pickle.load(f)

import json
with open(test_path, 'r') as f:
    test_data = json.load(f)

print(f"Train users: {len(df_train)}")
print(f"Test users: {len(test_data)}")
print(f"Num items: {NUM_ITEMS}")


[1/5] Cargando datos...
Train users: 16000
Test users: 1600
Num items: 752


In [10]:
from src.data.preprocessing import create_dt_dataset, DTDataset

# === 2. CREAR DATASET ===
print("\n[2/5] Creando dataset...")

max_len = 20  # longitud máxima de secuencia
train_data = create_dt_dataset(df_train, max_len=max_len)
train_dataset = DTDataset(train_data, max_len=max_len)

train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

print(f"Dataset size: {len(train_dataset)}")


[2/5] Creando dataset...


Creating dataset: 100%|██████████| 16000/16000 [00:01<00:00, 12762.80it/s]

Dataset size: 16000



/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [11]:
from src.models.decision_transformer import DecisionTransformerHF

# === 3. CREAR MODELO ===
print("\n[3/5] Creando modelo...")

model = DecisionTransformerHF(
    num_items=NUM_ITEMS,
    num_groups=NUM_GROUPS,
    hidden_dim=128,
    n_layers=3,
    n_heads=4,
    max_seq_len=max_len,
    dropout=0.1
)

total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")


[3/5] Creando modelo...
Total parameters: 803,056


In [12]:
from src.training.trainer import train_decision_transformer_hf

# === 4. ENTRENAR ===
print("\n[4/5] Entrenando modelo...")

model = train_decision_transformer_hf(
    model=model,
    train_loader=train_loader,
    num_epochs=10,
    learning_rate=1e-4,
    weight_decay=1e-4,
    device=device
)

# Guardar modelo
torch.save(model.state_dict(), f'decision_transformer_hf_{DATASET}.pt')
print(f"Modelo guardado: decision_transformer_hf_{DATASET}.pt")


[4/5] Entrenando modelo...


Epoch 1/10:   0%|          | 0/250 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 1/10:   0%|          | 0/250 [00:00<?, ?it/s]


AcceleratorError: CUDA error: device-side assert triggered
Search for `cudaErrorAssert' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
